In [10]:
for k in example.feature_lists.feature_list.keys():
    print(k)

RGB/feature/timestamp
AUDIO/feature/floats
RGB/feature/floats
AUDIO/feature/timestamp


In [14]:
a = example.feature_lists.feature_list["RGB/feature/floats"].feature

In [21]:
a[0].float_list.value

1024

In [80]:
import subprocess
import shlex
import os
import tempfile

In [81]:
def run_process(cmd: str):
    return subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE)

In [82]:
temp_dir = tempfile.TemporaryDirectory()

In [83]:
# Start Docker container
SHARED_FOLDER = temp_dir.name

cmd = f"docker run -dit -v {SHARED_FOLDER}:/v_folder --name mediapipe mediapipe:latest"
subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE)

CompletedProcess(args=['docker', 'run', '-dit', '-v', '/tmp/tmptzeuev0m:/v_folder', '--name', 'mediapipe', 'mediapipe:latest'], returncode=0, stdout=b'699fc0d213b05c84b7d7106f3bac395d09bb4397c8bace28899deaf7ff428151\n')

In [84]:
# Download youtube video

video_out = os.path.join(temp_dir.name, 'vid.mp4')

url = "https://www.youtube.com/watch?v=ico-nsXqvCg"
cmd = f"youtube-dl -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' '{url}' -o {video_out}"
run_process(cmd)

CompletedProcess(args=['youtube-dl', '-f', 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'https://www.youtube.com/watch?v=ico-nsXqvCg', '-o', '/tmp/tmptzeuev0m/vid.mp4'], returncode=0, stdout=b'[youtube] ico-nsXqvCg: Downloading webpage\n[download] Destination: /tmp/tmptzeuev0m/vid.f137.mp4\n\r\x1b[K[download]   0.0% of 20.10MiB at 989.69KiB/s ETA 00:21\r\x1b[K[download]   0.0% of 20.10MiB at  2.60MiB/s ETA 00:07\r\x1b[K[download]   0.0% of 20.10MiB at  5.64MiB/s ETA 00:03\r\x1b[K[download]   0.1% of 20.10MiB at 11.21MiB/s ETA 00:01\r\x1b[K[download]   0.2% of 20.10MiB at  7.85MiB/s ETA 00:02\r\x1b[K[download]   0.3% of 20.10MiB at  4.11MiB/s ETA 00:04\r\x1b[K[download]   0.6% of 20.10MiB at  4.91MiB/s ETA 00:04\r\x1b[K[download]   1.2% of 20.10MiB at  4.30MiB/s ETA 00:04\r\x1b[K[download]   2.5% of 20.10MiB at  5.34MiB/s ETA 00:03\r\x1b[K[download]   5.0% of 20.10MiB at  6.99MiB/s ETA 00:02\r\x1b[K[download]   9.9% of 20.10MiB at  6.79MiB/s ETA 00:02\r\x1b[K[download]  19.9% of 20.10Mi

In [89]:
# Build and run the inference on the provided file

cmd = f"docker exec mediapipe python -m mediapipe.examples.desktop.youtube8m.generate_input_sequence_example \
  --path_to_input_video=/v_folder/vid.mp4 \
  --clip_end_time_sec=600"

run_process(cmd)

CompletedProcess(args=['docker', 'exec', 'mediapipe', 'python', '-m', 'mediapipe.examples.desktop.youtube8m.generate_input_sequence_example', '--path_to_input_video=/v_folder/vid.mp4', '--clip_end_time_sec=600'], returncode=0, stdout=b'')

In [90]:
cmd = "docker exec mediapipe bazel build -c opt --linkopt=-s \
  --define MEDIAPIPE_DISABLE_GPU=1 --define no_aws_support=true \
  mediapipe/examples/desktop/youtube8m:extract_yt8m_features"
run_process(cmd)

CompletedProcess(args=['docker', 'exec', 'mediapipe', 'bazel', 'build', '-c', 'opt', '--linkopt=-s', '--define', 'MEDIAPIPE_DISABLE_GPU=1', '--define', 'no_aws_support=true', 'mediapipe/examples/desktop/youtube8m:extract_yt8m_features'], returncode=0, stdout=b'')

In [91]:
cmd = "docker exec mediapipe bazel-bin/mediapipe/examples/desktop/youtube8m/extract_yt8m_features \
  --calculator_graph_config_file=mediapipe/graphs/youtube8m/feature_extraction.pbtxt \
  --input_side_packets=input_sequence_example=/tmp/mediapipe/metadata.pb  \
  --output_side_packets=output_sequence_example=/v_folder/features.pb"

run_process(cmd)

CompletedProcess(args=['docker', 'exec', 'mediapipe', 'bazel-bin/mediapipe/examples/desktop/youtube8m/extract_yt8m_features', '--calculator_graph_config_file=mediapipe/graphs/youtube8m/feature_extraction.pbtxt', '--input_side_packets=input_sequence_example=/tmp/mediapipe/metadata.pb', '--output_side_packets=output_sequence_example=/v_folder/features.pb'], returncode=0, stdout=b'')

# Reformat extracted file to necessary format

In [116]:
import tensorflow as tf
import numpy as np

features_file = os.path.join(temp_dir.name, 'features.pb')

sequence_example = open(features_file, 'rb').read()
example = tf.train.SequenceExample.FromString(sequence_example)

In [117]:
for k in example.feature_lists.feature_list.keys():
    print(k)

RGB/feature/timestamp
AUDIO/feature/floats
RGB/feature/floats
AUDIO/feature/timestamp


In [133]:
a = example.feature_lists.feature_list["RGB/feature/floats"].feature

In [134]:
b = example.feature_lists.feature_list["AUDIO/feature/floats"].feature

In [123]:
a = np.array(list((l.float_list.value for l in a)))

In [125]:
b = np.array(list((l.float_list.value for l in b)))

In [131]:
mean_rgb = np.mean(a, axis=0)

In [132]:
mean_audio = np.mean(b, axis=0)

In [140]:
mean_audio

(128,)

In [162]:
np.array([mean_rgb]).shape

(1, 1024)

In [163]:
np.array([mean_audio]).shape

(1, 128)